In [1]:
import io, os
import requests
from PIL import Image
import torch
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets
import numpy as np
import cv2
import pdb

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, 257) # Modify the layer

finalconv_name = 'layer4'
model_ft.eval()

model_ft.load_state_dict(torch.load("test_model.pth",map_location='cpu')) # use my model parameter and use cpu

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize((224,224)),
   transforms.ToTensor(),
   normalize
])

features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

model_ft._modules.get(finalconv_name).register_forward_hook(hook_feature)

# get the softmax weight
params = list(model_ft.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [5]:
img_pil = Image.open("data/test1(octopus).jpg") # you can change image at here

img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = model_ft(img_variable)

h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

In [6]:
org_img = np.array(img_pil)

In [21]:
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

# render the CAM and output
org_img = np.array(img_pil,dtype=np.uint8)
org_img = org_img[:, :, ::-1].copy()
height, width, _ = org_img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + org_img * 0.5
cv2.imwrite('results/CAM1(octopus).jpg', result) # save result

True

In [36]:
# check result by opencv 
norm_result = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
concat_image = np.hstack((org_img,norm_result))
concat_image = cv2.resize(concat_image, dsize=(0, 0), fx=0.5, fy=0.5)
cv2.imshow('result',concat_image)
cv2.waitKey(0)
cv2.destroyAllWindows()